# Probability Model

## Create

In [1]:
file = open("./dataset/train.txt")

word_counter = {}
pair_counter = {}
vocabulary = set()
N = 0

for line in file.readlines():
    words = line.split()
    N += len(words)
    
    for i, word in enumerate(words):
        vocabulary.add(word)
        if word_counter.get(word) is None:
            word_counter[word] = 0
        word_counter[word] += 1
    
        if i>0:
            pair = (words[i-1], word)
            if pair_counter.get(pair) is None:
                pair_counter[pair] = 0
            pair_counter[pair] += 1

In [2]:
def unigram(word):
    return word_counter[word]/N

In [3]:
def create_bigram(sigma):
    def calculate_B(sigma):
        B_word = {}
        for word1 in vocabulary:
            B_word[word1] = len(vocabulary)

        for pair in pair_counter:
            word1 = pair[0]
            B_word[word1] -= 1
        
        return B_word
    
    B_word = calculate_B(sigma)
    
    def bigram(word1, word2):
        alpha = sigma / word_counter[word1] * B_word[word1]
        bigram_probability_item = {}
        p_bg = word_counter[word1] / N

        return (max(pair_counter.get((word1, word2), 0) - sigma, 0) /word_counter[word1]) + alpha * p_bg

    return bigram

In [4]:
bigram = create_bigram(sigma=0.1)

In [5]:
bigram(word1="من", word2= "ایران")

0.004385964087839584

In [11]:
bigram(word1="از", word2= "من")

0.016205333998990382

## Fine-tuning

## Perplexity

In [27]:
import math

def calculate_unigram_perplexity():
    H = 0
    
    for word in vocabulary:
#         p_word = word_counter[word]/N
        p_word = unigram(word)
        H -= p_word * math.log(p_word , 2)
        
    return 2**H

def calculate_bigram_perplexity():
    H = 0
    
    for i, word1 in enumerate(vocabulary):
        if i%1000==0:
            print(i)
        for word2 in vocabulary:
            p_pair = bigram(word1, word2)
            H -= p_pair * math.log(p_pair , 2)
        
    return 2**H
    

In [28]:
calculate_unigram_perplexity()

1768.1731873435156

In [29]:
calculate_bigram_perplexity()

0
1000


KeyboardInterrupt: 

## Test Unigram

In [75]:
best_word = None
best_probability = -math.inf
for condidate_word in vocabulary:
    if unigram(condidate_word) > best_probability:
        best_probability = unigram(condidate_word)
        best_word = condidate_word

file = open("./dataset/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("./dataset/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    print(line.strip(), end=" ")
    for _ in range(digit):
        print(f'"{best_word}"', end=" ")
    print()
    
    print(incomplete_gold_lines[i])    

این سخن حقست اگر نزد سخن گستر "و" 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد "و" 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با "و" 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است "و" 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را "و" "و" 
آتش ابراهیم را نبود زیان

من که اندر سر "و" "و" 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش "و" "و" 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به "و" "و" "و" 
هرکه از حق به سوی او نظریست

گفت این از "و" "و" "و" 
گفت این از خدای باید خواست

کلاه لاله که لعل است "و" "و" "و" 
کلاه لاله که لعل است اگر تو بشناسی


## Test Bigram

In [76]:
file = open("./dataset/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("./dataset/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    line = line.strip()
    words = line.split()
    
    last_word = words[-1]
    
    print(line, end=" ")
    
    for _ in range(digit):        
        best_word = None
        best_probability = -math.inf
        
        for condidate_word in vocabulary:
            if bigram(last_word, condidate_word) > best_probability:
                best_probability = bigram(last_word, condidate_word)
                best_word = condidate_word
            
        print(f'"{best_word}"', end=" ")
        last_word = best_word
    
    print()
    
    print(incomplete_gold_lines[i])  

این سخن حقست اگر نزد سخن گستر "خویش" 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد "کرد" 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با "تو" 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است "و" 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را "به" "دست" 
آتش ابراهیم را نبود زیان

من که اندر سر "و" "از" 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش "و" "از" 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به "دست" "و" "از" 
هرکه از حق به سوی او نظریست

گفت این از "آن" "را" "به" 
گفت این از خدای باید خواست

کلاه لاله که لعل است "و" "از" "آن" 
کلاه لاله که لعل است اگر تو بشناسی
